In [8]:

!pip install chart-studio

import chart_studio.plotly as py
import numpy as np 
import pandas as pd 
import sqlite3
from collections import Counter
import nltk
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud, STOPWORDS


# Plotly libraries
import plotly.tools as tls
import chart_studio.plotly as py
import plotly.figure_factory as ff
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

import os



In [9]:
df = pd.read_csv("consumer_complaints.csv",low_memory=False)
df.head()

,date_received,product,sub_product,issue,sub_issue,consumer_complaint_narrative,company_public_response,company,state,zipcode,tags,consumer_consent_provided,submitted_via,date_sent_to_company,company_response_to_consumer,timely_response,consumer_disputed?,complaint_id
0,08/30/2013,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",NaN,NaN,NaN,U.S. Bancorp,CA,95993,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511074
1,08/30/2013,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",NaN,NaN,NaN,Wells Fargo & Company,CA,91104,NaN,NaN,Referral,09/03/2013,Closed with explanation,Yes,Yes,511080
2,08/30/2013,Credit reporting,NaN,Incorrect information on credit report,Account status,NaN,NaN,Wells Fargo & Company,NY,11764,NaN,NaN,Postal mail,09/18/2013,Closed with explanation,Yes,No,510473
3,08/30/2013,Student loan,Non-federal student loan,Repaying your loan,Repaying your loan,NaN,NaN,"Navient Solutions, Inc.",MD,21402,NaN,NaN,Email,08/30/2013,Closed with explanation,Yes,Yes,510326
4,08/30/2013,Debt collection,Credit card,False statements or representation,Attempted to collect wrong amount,NaN,NaN,Resurgent Capital Services L.P.,GA,30106,NaN,NaN,Web,08/30/2013,Closed with explanation,Yes,Yes,511067


In [10]:
dispute_presence = df.loc[df['consumer_disputed?'] == 'Yes']
cross_month = pd.crosstab(dispute_presence['state'], dispute_presence['company']).apply(lambda x: x/x.sum() * 100)

In [11]:
#  Share of Most disputes for Bank of America.
df_boa = pd.DataFrame(cross_month['Bank of America']).reset_index().sort_values(by="Bank of America", ascending=False).round(2)
df_boa = df_boa.rename(columns={'Bank of America': 'share of complaints'})

for col in df_boa.columns:
    df_boa[col] = df_boa[col].astype(str)
    
    
scl = [[0.0, 'rgb(202, 202, 202)'],[0.2, 'rgb(253, 205, 200)'],[0.4, 'rgb(252, 169, 161)'],
            [0.6, 'rgb(247, 121, 108  )'],[0.8, 'rgb(255, 39, 39)'],[1.0, 'rgb(219, 0, 0)']]


df_boa['text'] = "State Code: " + df_boa['state'] + '<br>'


data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df_boa['state'],
        z = df_boa['share of complaints'], 
        locationmode = 'USA-states',
        text = df_boa['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "%")
        ) ]

layout = dict(
    title = 'Most Complaints by State <br> Bank of America',
    geo = dict(
        scope = 'usa',
        projection=dict(type='albers usa'),
        showlakes = True,
        lakecolor = 'rgb(255, 255, 255)')
)

fig = dict(data=data, layout=layout)
iplot(fig, filename='d3-cloropleth-map')

In [12]:
import pandas as pd
import plotly.graph_objs as go
from plotly.offline import iplot
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output

# Load the data
df = pd.read_csv("consumer_complaints.csv",low_memory=False)
import dash
print(f"Dash Version: {dash.__version__}")

import dash_core_components as dcc
print(f"Dash Core Components Version: {dcc.__version__}")

Dash Version: 2.16.1
Dash Core Components Version: 2.13.1


In [13]:
def create_choropleth_map(company):
    # Filter the data for the selected company and disputes
    dispute_presence = df.loc[(df['consumer_disputed?'] == 'Yes') & (df['company'] == company)]

    # Calculate the share of complaints by state
    cross_month = pd.crosstab(dispute_presence['state'], dispute_presence['company']).apply(lambda x: x / x.sum() * 100)
    df_complaints = pd.DataFrame(cross_month[company]).reset_index().sort_values(by=company, ascending=False).round(2)
    df_complaints = df_complaints.rename(columns={company: 'share of complaints'})

    for col in df_complaints.columns:
        df_complaints[col] = df_complaints[col].astype(str)

    scl = [[0.0, 'rgb(202, 202, 202)'], [0.2, 'rgb(253, 205, 200)'], [0.4, 'rgb(252, 169, 161)'],
           [0.6, 'rgb(247, 121, 108)'], [0.8, 'rgb(255, 39, 39)'], [1.0, 'rgb(219, 0, 0)']]

    df_complaints['text'] = "State Code: " + df_complaints['state'] + '<br>'

    data = [dict(
        type='choropleth',
        colorscale=scl,
        autocolorscale=False,
        locations=df_complaints['state'],
        z=df_complaints['share of complaints'],
        locationmode='USA-states',
        text=df_complaints['text'],
        marker=dict(
            line=dict(
                color='rgb(255,255,255)',
                width=2
            )
        ),
        colorbar=dict(
            title="%"
        )
    )]

    layout = dict(
        title=f'Most Complaints by State <br> {company}',
        geo=dict(
            scope='usa',
            projection=dict(type='albers usa'),
            showlakes=True,
            lakecolor='rgb(255, 255, 255)'
        )
    )

    fig = dict(data=data, layout=layout)
    return fig

In [14]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1('Customer Disputes Dashboard'),
    html.Div([
        dcc.Dropdown(
            id='company-dropdown',
            options=[{'label': company, 'value': company} for company in df['company'].unique()],
            value=df['company'].unique()[0]  # Set the default value to the first unique company
        )
    ]),
    dcc.Graph(id='choropleth-map')
])

@app.callback(
    Output('choropleth-map', 'figure'),
    [Input('company-dropdown', 'value')]
)
def update_choropleth_map(company):
    fig = create_choropleth_map(company)
    return fig

if __name__ == '__main__':
    app.run_server(debug=True)

Address already in use
Port 8050 is in use by another program. Either identify and stop that program, or start the server with a different port.


AttributeError: 'tuple' object has no attribute 'tb_frame'